In [6]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torchvision.models as models
import string
import random

%matplotlib inline

In [7]:
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt
import numpy as np
import time
from joblib import dump, load
from PlotToImage import PlotToImage
from auto_tsmo import full_method
import glob
import json

In [8]:
dataset_path = "C://Users//admin//Downloads//FLIR_0//FLIR_ADAS_1_3/"
import glob
data = [cv2.imread(file) for file in glob.glob('C://Users//admin//Google//SomeData//PreviewData//' + "*.jpeg")]

In [9]:
class HistogramsDataset(Dataset):
    def __init__(self, path, values, transform=None):
   #     self.path = dataset_path + 'train/thermal_8_bit/'
        self.images = [cv2.imread(file, 0) for file in glob.glob(path + "*.jpeg")]
        self.values = values
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        sample = {'image': self.images[idx], 'vals': self.values[idx]}
        if self.transform:
            sample = self.transform(sample)
        return sample
    
class Transformation(object):
    def __call__(self, sample):
        image, vals = sample['image'], sample['vals']

        return {'image': image,
                'vals': vals}

In [10]:
IMAGE_WIDTH = 640
with open(dataset_path + "/train/thermal_annotations.json", "r") as read_file:
    train_data = json.load(read_file)
train_values = np.array([np.zeros(IMAGE_WIDTH) for i in range(8862)], dtype=np.uint8)
for ind in range(0, train_data['annotations'].__len__()):
    d = train_data['annotations'][ind]
    img_num = d['image_id']
    vals = train_values[img_num]
    if d['category_id'] == 1:
        x,y,w,h  = d['bbox']
        for i in range(x, x + w):
            vals[i] = 1

In [ ]:
train_values

In [11]:
train_dataset = HistogramsDataset(dataset_path + 'train/thermal_8_bit/', train_values, transform=Transformation())

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [16]:
with open(dataset_path + "/val/thermal_annotations.json", "r") as read_file:
    test_data = json.load(read_file)
test_data

{'info': {'version': 1.0,
  'url': 'no url specified',
  'year': 2019,
  'date_created': 'today',
  'contributor': 'no contributor specified',
  'description': ''},
 'categories': [{'name': 'person', 'id': 1, 'supercategory': 'unknown'},
  {'name': 'bicycle', 'id': 2, 'supercategory': 'unknown'},
  {'name': 'car', 'id': 3, 'supercategory': 'unknown'},
  {'name': 'motorcycle', 'id': 4, 'supercategory': 'unknown'},
  {'name': 'airplane', 'id': 5, 'supercategory': 'unknown'},
  {'name': 'bus', 'id': 6, 'supercategory': 'unknown'},
  {'name': 'train', 'id': 7, 'supercategory': 'unknown'},
  {'name': 'truck', 'id': 8, 'supercategory': 'unknown'},
  {'name': 'boat', 'id': 9, 'supercategory': 'unknown'},
  {'name': 'traffic light', 'id': 10, 'supercategory': 'unknown'},
  {'name': 'fire hydrant', 'id': 11, 'supercategory': 'unknown'},
  {'name': 'stop sign', 'id': 12, 'supercategory': 'unknown'},
  {'name': 'parking meter', 'id': 13, 'supercategory': 'unknown'},
  {'name': 'bench', 'id': 14, 

In [17]:
test_data['annotations'].__len__()

11696

In [18]:
%%time
test_histograms = []
for img_num in range(8862, 10228):
    img = cv2.imread(dataset_path + 'val/thermal_8_bit/FLIR_' +
                 "".join(['0' for i in [img_num + 1 - (img_num + 1) % (10 ** j) for j in range(5)] if not i]) +
                 str(img_num + 1) +
                 '.jpeg', 0)
    hist = np.array([img[:,i].sum() for i in range(img.shape[1])])
    hist = hist / hist.max()
    test_histograms.append(hist)
test_histograms = np.array(test_histograms)

Wall time: 12.3 s


In [20]:
test_histograms.shape

(1366, 640)

In [21]:
%%time
test_values = np.array([np.zeros(test_histograms.shape[1]) for i in range(8862, 10228)], dtype=np.uint8)

for ind in range(0, test_data['annotations'].__len__()):
    d = test_data['annotations'][ind]
    img_num = d['image_id']
    vals = test_values[img_num]
    if d['category_id'] == 1:
        x,y,w,h  = d['bbox']
        for i in range(x, x + w):
            vals[i] += 1

Wall time: 813 ms


In [23]:
test_values.shape

(1366, 640)

In [24]:
[vals for vals in test_values if not vals.max()].__len__()

160

In [343]:
test_dataset = HistogramsDataset(test_histograms, test_values, transform=Transformation())

In [344]:
trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
print(len(trainloader), len(trainloader.dataset))
for i_batch, sample_batched in enumerate(trainloader):
    ___hist = sample_batched['histogram']
    print(i_batch, sample_batched['histogram'].size(),
          sample_batched['vals'].size())
    if i_batch == 3:
        break

139 8862
0 torch.Size([64, 640]) torch.Size([64, 640])
1 torch.Size([64, 640]) torch.Size([64, 640])
2 torch.Size([64, 640]) torch.Size([64, 640])
3 torch.Size([64, 640]) torch.Size([64, 640])


In [345]:
testloader = DataLoader(test_dataset, batch_size=64, shuffle=True)
print(len(testloader), len(testloader.dataset))
for i_batch, sample_batched in enumerate(testloader):
    print(i_batch, sample_batched['histogram'].size(),
          sample_batched['vals'].size())
    if i_batch == 3:
        break

22 1366
0 torch.Size([64, 640]) torch.Size([64, 640])
1 torch.Size([64, 640]) torch.Size([64, 640])
2 torch.Size([64, 640]) torch.Size([64, 640])
3 torch.Size([64, 640]) torch.Size([64, 640])


In [346]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [347]:
def Train(model, train_loader, optimizer):
    model.train()
    
    processed = 0
    total = len(train_loader.dataset)
    
    for batch_idx, sample_batched in enumerate(train_loader):
        X, y = sample_batched['histogram'], sample_batched['vals']
        X, y = X.to(device), y.to(device)#.long()
       # print(X, X.size(), sep='\n')
       # print(y, y.size(), sep='\n')
        optimizer.zero_grad()
        probs = net(X)
       # print(probs, probs.size(), sep='\n')
        
        loss = torch.sum(torch.norm((y - probs).view(64, -1), p=2, dim=1))

        #loss = criterion(probs, y)
        
      #  with torch.no_grad():
     #       print(np.array([LA.norm(X[i]-probs[i]) for i in range(len(probs))]).sum(), loss)
        
        loss.backward()
        optimizer.step()      
        
        processed += len(X)
        
        if not batch_idx % 10:
            print(f'{processed}/{total}, loss: {loss.item()}')

In [348]:
from numpy import linalg as LA
def Test(model, test_loader):
    with torch.no_grad():
        model.eval()
        processed = 0
        total = len(test_loader.dataset)
        sum = 0
        new_sum = 0
        for batch_idx, sample_batched in enumerate(test_loader):
            X, y = sample_batched['histogram'], sample_batched['vals']
            X, y = X.to(device), y.to(device).long()
            probs = net(X)

            s = np.array([torch.all(torch.eq(X[i], probs[i])) for i in range(len(probs))]).sum()
            new_s = int(np.array([LA.norm(X[i]-probs[i]) for i in range(len(probs))]).sum())
            processed += len(X)
            sum += s
            new_sum += new_s
            if not batch_idx % 100:
                print(sum/processed)
                print(new_sum)
        print(sum, '/', total, sum/total)

In [349]:
class Net(nn.Module):
    def __init__(self, input_output_size):
        super(Net, self).__init__()
        self.size = input_output_size
        self.neurons_num = self.size * 2
        self.fc1 = nn.Linear(self.size, self.neurons_num)
        self.fc2 = nn.Linear(self.neurons_num, self.neurons_num)
        self.fc3 = nn.Linear(self.neurons_num, self.neurons_num)
        self.fc4 = nn.Linear(self.neurons_num, self.size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

net = Net(640).to(device)
name = 'net'
net.double()

Net(
  (fc1): Linear(in_features=640, out_features=1280, bias=True)
  (fc2): Linear(in_features=1280, out_features=1280, bias=True)
  (fc3): Linear(in_features=1280, out_features=1280, bias=True)
  (fc4): Linear(in_features=1280, out_features=640, bias=True)
)

In [350]:
#net = torch.load('vgg2epochs')

In [351]:
#net(___hist[0])

In [352]:
epoch_n = 5
criterion = nn.MultiLabelSoftMarginLoss()
rates = [1e-4, 1e-6, 1e-6, 1e-6, 1e-7]

for epoch in range(0, epoch_n):
    learning_rate = rates[epoch]
    optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
    print(f'Epoch {epoch}:')
    Train(net, trainloader, optimizer)
    torch.save(net, f'{name}{epoch}epochs')
    Test(net, testloader)

Epoch 0:
64/8862, loss: 318.2495100547151
704/8862, loss: 390.9197058415016
1344/8862, loss: 286.8354134204559
1984/8862, loss: 302.04721330090626
2624/8862, loss: 256.9790909646639
3264/8862, loss: 342.5306779017584
3904/8862, loss: 320.6958788872937
4544/8862, loss: 347.0010453942726
5184/8862, loss: 297.858521426655
5824/8862, loss: 293.91983446598647
6464/8862, loss: 290.42075601709735
7104/8862, loss: 313.2341552869103
7744/8862, loss: 350.2350813933536
8384/8862, loss: 390.0001544046993
0.0
1335
0 / 1366 0.0
Epoch 1:
64/8862, loss: 344.91324052443053
704/8862, loss: 363.4860889270025
1344/8862, loss: 297.17135943119763
1984/8862, loss: 365.7697941604854


KeyboardInterrupt: 

In [353]:
from numpy import linalg as LA
def Test(model, test_loader):
    with torch.no_grad():
        model.eval()
        processed = 0
        total = len(test_loader.dataset)
        for batch_idx, sample_batched in enumerate(test_loader):
            X, y = sample_batched['histogram'], sample_batched['vals']
            X, y = X.to(device), y.to(device).long()
            probs = net(X)
            for i in range(len(probs)):
                print(probs[i])
                print(y[i])
            break

In [354]:
%%time
Test(net, testloader)

tensor([ 4.2721e-04,  1.9678e-04,  3.4694e-04,  4.8732e-04,  5.4428e-04,
         3.1462e-04,  6.4752e-04,  1.1984e-04,  1.5726e-04,  9.3089e-04,
         3.2781e-04,  1.0530e-03, -8.1488e-05,  1.2141e-03,  7.6342e-04,
         3.9187e-04,  1.0761e-03,  2.5626e-04,  1.1301e-04,  9.1625e-04,
        -4.8932e-04,  3.8518e-04,  4.3487e-04,  4.5630e-04,  1.1943e-03,
        -8.7269e-06, -2.4083e-04,  5.8700e-04,  7.6857e-04,  4.7831e-04,
         2.1847e-04,  4.2008e-04,  6.0310e-04,  3.5259e-05,  4.3781e-04,
         7.8278e-04,  6.7290e-04,  9.3030e-05,  5.3324e-04, -5.7732e-04,
         5.7461e-04,  1.4110e-03,  5.9122e-04, -3.5268e-04,  8.6434e-04,
         1.1091e-03,  4.4167e-04,  6.4679e-05, -4.7809e-05, -1.6526e-04,
         6.2059e-04,  9.7882e-04,  1.7006e-04,  3.8367e-04,  6.2171e-04,
         4.1903e-04,  5.3497e-04,  5.3554e-04,  2.3683e-04, -1.2019e-04,
        -4.3235e-04,  1.1071e-03,  1.9398e-04,  3.3526e-04, -2.3106e-04,
         1.2655e-04, -1.3099e-04,  7.5101e-04,  2.6

        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([-2.4703e-04, -4.9344e-04,  1.2830e-04,  3.6335e-04,  3.0927e-05,
         2.1577e-03,  9.6122e-04,  1.1833e-03,  4.6931e-04, -5.1011e-04,
         1.6421e-04,  3.0865e-04,  4.8498e-04,  5.5450e-04, -5.2571e-04,
         1.5057e-03,  5.2639e-04, -2.0519e-04,  4.8939e-04,  3.9275e-04,
         1.7772e-04,  1.0953e-03,  6.0572e-04,  3.3912e-04,  7.5825e-04,
        -4.6946e-04,  6.4766e-04,  1.1194e-03,  2.2094e-03, -2.4724e-04,
         6.0809e-04,  6.6245e-04, -3.7247e-04, -6.6739e-05,  1.8028e-03,
         3.1885e-04,  9.7748e-04,  1.0999e-04,  6.9694e-04,  1.3251e-03,
         7.5332e-04,  1.5025e-04,  9.8933e-04, -5.0907e-04,  1.3329e-04,
        -6.6647e-04, -9.1546e-05,  7.3119e-04, -2.4705e-04,  3.4797e-04,
         8.5507e-04, -9.0397e-04,  7.7409e-04,  2.3136e-04, -5.3795e-04,
        -1.2540e-04,  7.4282e-04,  4.7970e-04,  2.4309e-04, -6.7402e-04,
         7.3504e-04,  2.7044e-04,  1.6133e-03, -9.5027e-04, -9.8947

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

tensor([ 4.8025e-04,  4.2781e-04,  1.1015e-03, -1.0205e-04,  2.4615e-04,
         2.6861e-04,  6.6817e-04,  2.3777e-04,  6.9424e-04, -4.1733e-04,
         2.3420e-04,  1.1160e-03,  3.1720e-04, -4.3464e-05,  1.4585e-04,
         9.9098e-04,  1.4346e-04,  7.8938e-04, -4.0722e-04,  3.4131e-04,
        -3.3075e-04, -1.3630e-03,  4.7065e-04,  8.4671e-04,  6.4698e-04,
        -2.6452e-04,  4.2285e-04,  1.4871e-03,  5.4441e-04, -1.2336e-04,
        -3.8691e-04,  1.5374e-03,  1.3061e-03,  5.1883e-05,  1.6356e-04,
         1.0419e-03,  2.5668e-03,  3.6681e-04,  1.1515e-03,  2.9305e-04,
         1.7775e-03, -8.8428e-04,  1.9575e-04,  8.6677e-04,  7.9007e-04,
         8.5520e-04,  3.2959e-04,  4.6733e-04,  2.5447e-04,  1.4806e-03,
         1.3013e-03,  1.9194e-03, -2.9994e-04,  8.9989e-04,  1.4221e-03,
         1.9477e-03,  1.5324e-03,  4.5838e-04, -7.7610e-04,  2.3147e-03,
         4.1931e-04, -9.8332e-05, -4.4972e-04, -4.6857e-05,  5.1886e-04,
         1.5226e-03,  1.3478e-03,  1.9328e-03, -8.1

        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([-5.8525e-04,  2.1822e-04, -2.6467e-05,  7.4231e-04, -1.4579e-04,
         5.2138e-04, -1.2338e-04,  7.9872e-04, -6.2600e-04,  9.7555e-04,
         1.6420e-04,  9.3928e-04, -5.2138e-04, -7.7851e-05,  9.2146e-05,
        -7.6084e-05,  1.1527e-03, -2.9271e-04,  4.2680e-04,  5.2506e-04,
         2.9702e-04,  2.3422e-04,  2.5596e-04, -9.6366e-05, -1.0782e-04,
         1.2287e-03, -8.6834e-04,  7.5223e-04, -4.4524e-04, -7.2608e-04,
         1.1453e-03, -1.2961e-04, -6.8074e-04, -3.4398e-04,  3.3761e-05,
        -7.4298e-04, -1.8046e-04,  4.1512e-04,  6.1858e-04, -6.9771e-05,
         2.1980e-04,  2.0755e-03,  7.4565e-04,  2.9528e-05,  3.7873e-04,
        -1.2164e-04,  4.6074e-04,  3.2026e-04, -2.6221e-04,  1.2944e-03,
         2.6195e-04,  2.6798e-04,  4.3788e-04, -3.1559e-04,  1.5671e-04,
         4.8886e-04, -6.6827e-06,  4.2616e-04,  1.1473e-03, -1.0501e-03,
         3.5599e-04,  9.5282e-04,  3.9385e-05,  1.0502e-03, -6.6842

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

tensor([ 4.1298e-04,  1.6527e-03,  4.0296e-04, -8.5903e-04,  7.2166e-05,
         6.7557e-04,  6.0743e-04, -3.9607e-04,  7.0872e-05, -2.7993e-04,
         2.2145e-04, -3.7492e-04,  1.0725e-03, -4.2154e-04, -4.4019e-04,
         3.6644e-04,  8.4531e-04,  1.8949e-03, -2.5549e-04,  1.0980e-03,
         4.2550e-04,  7.0399e-04, -2.8591e-04,  1.5088e-03,  2.0277e-03,
         6.8725e-04,  2.5891e-04,  1.2853e-03,  7.1618e-04,  2.4983e-04,
        -3.7249e-04,  1.9105e-03,  5.4314e-04,  1.6924e-04,  1.7775e-03,
         2.1500e-03,  4.2902e-04,  2.3123e-04,  9.0358e-04,  1.7060e-03,
         1.1140e-03,  6.0451e-04, -1.8218e-04, -2.2609e-05,  2.6804e-03,
         4.3308e-04,  2.8767e-03,  1.3655e-03,  4.9175e-05,  1.5801e-03,
         7.3112e-04,  1.5643e-03, -1.5629e-04,  4.5767e-04,  3.3711e-03,
         8.4329e-04,  4.2821e-04,  1.0078e-03,  5.4382e-04,  4.9296e-04,
         1.5081e-03,  5.9179e-04,  1.5213e-03,  3.7372e-04,  1.1886e-03,
         1.0356e-03,  4.4469e-04,  5.8270e-04, -2.5

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,